Resource consulted: https://skimai.com/fine-tuning-bert-for-sentiment-analysis/

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 14.4 MB/s 
     |████████████████████████████████| 3.3 MB 53.5 MB/s 
     |████████████████████████████████| 596 kB 72.9 MB/s 
     |████████████████████████████████| 895 kB 59.0 MB/s 
     |████████████████████████████████| 61 kB 624 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import files
CS_file = files.upload()

Saving CovidSentimentData.csv to CovidSentimentData.csv


In [ ]:
import io
import pandas as pd

data = pd.read_csv(io.BytesIO(CS_file['CovidSentimentData.csv']))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback


# Read data
#data = pd.read_csv("train.csv")

# Define pretrained tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model = model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, eps=1e-8)
lambda1 = lambda epoch: 0.65 ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda1)
optimizers = optimizer, scheduler

# ----- 1. Preprocess data -----#
# Preprocess data

X = list(data["Tweet"])
y = list(data["target"])

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=100,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    seed=19,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    #optimizers=optimizers,
)

# Train pre-trained model

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 15300
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 11475


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,No log,0.607305,0.827496,0.827496,1.000000,0.905606
200,No log,0.491077,0.827496,0.827496,1.000000,0.905606
300,No log,0.461858,0.827496,0.827496,1.000000,0.905606
400,No log,0.464268,0.827496,0.827496,1.000000,0.905606
500,0.530700,0.488258,0.827496,0.827496,1.000000,0.905606
600,0.530700,0.471610,0.827496,0.827496,1.000000,0.905606
700,0.530700,0.461960,0.827496,0.827496,1.000000,0.905606
800,0.530700,0.459857,0.827496,0.827496,1.000000,0.905606
900,0.530700,0.471234,0.827496,0.827496,1.000000,0.905606
1000,0.500600,0.510990,0.827496,0.827496,1.000000,0.905606


***** Running Evaluation *****
  Num examples = 3826
  Batch size = 4
***** Running Evaluation *****
  Num examples = 3826
  Batch size = 4
***** Running Evaluation *****
  Num examples = 3826
  Batch size = 4
***** Running Evaluation *****
  Num examples = 3826
  Batch size = 4
***** Running Evaluation *****
  Num examples = 3826
  Batch size = 4
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3826
  Batch size = 4
***** Running Evaluation *****
  Num examples = 3826
  Batch size = 4
***** Running Evaluation *****
  Num examples = 3826
  Batch size = 4
***** Running Evaluation *****
  Num examples = 3826
  Batch size = 4
***** Running Evaluation *****
  Num examples = 3826
  Batch size = 4
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in 

TrainOutput(global_step=2100, training_loss=0.5165888123285203, metrics={'train_runtime': 441.0912, 'train_samples_per_second': 104.06, 'train_steps_per_second': 26.015, 'total_flos': 561166547760000.0, 'train_loss': 0.5165888123285203, 'epoch': 0.55})

In [ ]:
from google.colab import files
PS_file = files.upload()

KeyboardInterrupt: ignored

In [ ]:
ps_data = pd.read_csv('/content/prediction_score.csv')

In [ ]:
# ----- 3. Predict -----#
# Load test data
test_data = ps_data

X_test = list(test_data["Tweet"])
y_test = list(test_data["target"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized, y_test)

# Load trained model

#model_path = "output/checkpoint-50000"
#model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)
#model = model.train()
#optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, eps=1e-8)

#model = model.eval()

#eps, alpha = optimizer.privacy_engine.get_privacy_spent(DELTA)

#raw_pred, _, _ = model.predict(X_test)
#y_pred = np.argmax(raw_pred, axis=1)

#print(y_pred)
# Define test trainer
#test_trainer = Trainer(test_model)

# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)


***** Running Prediction *****
  Num examples = 5
  Batch size = 4


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_pred

array([1, 1, 1, 1, 1])

In [ ]:
raw_pred

array([[-1.5001374 ,  0.74881935],
       [-1.5001309 ,  0.7488121 ],
       [-1.5001175 ,  0.7488279 ],
       [-1.500128  ,  0.74884063],
       [-1.5001231 ,  0.74882114]], dtype=float32)